In [1]:
import glob
import numpy as np
import pandas as pd
from astropy.time import Time
import cdflib
import datetime
import spacepy.datamodel as dm
from astropy.table import Table
import math
from scipy.optimize import curve_fit
from matplotlib import pyplot as plt
import tqdm



def earth_radius(B):
    B = math.radians(B)  # converting into radians
    a = 6378.137  # Radius at sea level at equator
    b = 6356.752  # Radius at poles
    c = (a ** 2 * math.cos(B)) ** 2
    d = (b ** 2 * math.sin(B)) ** 2
    e = (a * math.cos(B)) ** 2
    f = (b * math.sin(B)) ** 2
    R = math.sqrt((c + d) / (e + f))
    return R

def objective(x, a, b):
    return  a*np.exp(b*x)

def find_nearest_idx(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

spatial_window = 5
temporal_window = 15
altitude_window = 100

radar_list = glob.glob('../data/external/Madrigal/2018-2021/EISCATSvalbardISRadarLongyearbyen/*.hdf5')
radar_list.sort()

gf_list = glob.glob('../data/interim/GRACEFO/KBRNE_relative_v1/dat/*')
gf_list.sort()

headerlist = ['CDF Epoch', 'GPS', 'Latitude', 'Longitude', 'Radius', 'Latitude_QD', 'Longitude_QD',
              'MLT', 'GRACE_1_Position', 'GRACE_2_Position', 'Iono_Corr', 'Distance', 'Relative_Hor_TEC', 'Relative_Ne']

gf_list_datetimes = []
for gf_file in gf_list:
    gf = pd.read_csv(gf_file, sep='\s+', header=0, index_col=False, names=headerlist)
    gf_list_datetimes.append((Time(gf['CDF Epoch'].values[-1], format='cdf_epoch').datetime).strftime('%Y-%m-%d'))



def find_conjunctions(radar_file, gf_list, gf_list_datetimes):
    radar_select_0 = []
    gf_select_0 = []
    radar_select_1 = []
    gf_select_1 = []
    radar_select_2 = []
    gf_select_2 = []

    radar_metadata = []
    # gf_file = []
    # Re = []


    radar = (Table.read(radar_file, path='Data/Table Layout')).to_pandas()
    radar_date = datetime.datetime(int(radar.year[0]), int(radar.month[0]), int(radar.day[0])).strftime('%Y-%m-%d')

    date, idx_gf, idx_radar = np.intersect1d(gf_list_datetimes, radar_date, return_indices=True)

    if len(idx_radar) > 0:


        gf_file = gf_list[idx_gf[0]]
        gf = pd.read_csv(gf_file, sep='\s+', header=0, index_col=False, names=headerlist)
        gf['dates'] = (Time(gf['CDF Epoch'].values, format='cdf_epoch')).datetime

        radar_metadata = (Table.read(radar_file, path='Metadata/Experiment Parameters')).to_pandas()

        names = np.array([x.decode() for x in radar_metadata['name']])
        values = np.array([x.decode() for x in radar_metadata['value']])

        radar_metadata = pd.DataFrame({'name': names,
                                       'value': values})
        radar_lat = float(radar_metadata[radar_metadata['name'] == 'instrument latitude']['value'].values[0])
        radar_lon = float(radar_metadata[radar_metadata['name'] == 'instrument longitude']['value'].values[0])
        radar['dates'] = Time(radar[['ut1_unix', 'ut2_unix']].mean(axis=1), format='unix').datetime

        gf = gf[((gf['Latitude'] <= radar_lat + spatial_window) & (gf['Latitude'] >= radar_lat - spatial_window))]
        gf = gf[((gf['Longitude'] <= radar_lon + spatial_window) & (gf['Longitude'] >= radar_lon - spatial_window))]

        if len(gf) > 0:

            gf_pass = gf[gf.diff(axis=0)['dates'] > datetime.timedelta(minutes=temporal_window)]

            if len(gf_pass) == 0:  # only one passage

                time_window_i = gf['dates'][gf.index.min()] - datetime.timedelta(minutes=temporal_window)
                time_window_f = gf['dates'][gf.index.max()] + datetime.timedelta(minutes=temporal_window)

                radar['dates'] = Time(radar[['ut1_unix', 'ut2_unix']].mean(axis=1), format='unix').datetime

                radar_select_0 = radar[(radar['dates'] >= time_window_i) & (radar['dates'] <= time_window_f)]
                gf_select_0 = gf[(gf['dates'] >= time_window_i) & (gf['dates'] <= time_window_f)]

                if len(radar_select_0) > 0:
                    Re = earth_radius(radar_lat)
                    altitude = np.mean(gf_select_0['Radius'] * 0.001 - Re)

                    altitude_window_i = altitude - altitude_window
                    altitude_window_f = altitude + altitude_window

                    radar_select_0 = radar_select_0[
                        (radar_select_0['gdalt'] >= altitude_window_i) & (radar_select_0['gdalt'] <= altitude_window_f)]


            if len(gf_pass) == 1:  # two passages

                pass_idx = gf[gf['dates'] == gf_pass['dates'][gf_pass.index.min()]].index[0]

                # first one

                # define time window
                time_window_i = gf['dates'][gf.index.min()] - datetime.timedelta(minutes=15)
                time_window_f = gf['dates'][
                                    gf.index[np.argwhere(gf.index == pass_idx)[0][0] - 1]] + datetime.timedelta(
                    minutes=15)

                radar_select_1 = radar[(radar['dates'] >= time_window_i) & (radar['dates'] <= time_window_f)]
                gf_select_1 = gf[(gf['dates'] >= time_window_i) & (gf['dates'] <= time_window_f)]

                if len(radar_select_1) > 0:
                    Re = earth_radius(radar_lat)
                    altitude = np.mean(gf_select_1['Radius'] * 0.001 - Re)

                    altitude_window_i = altitude - altitude_window
                    altitude_window_f = altitude + altitude_window

                    radar_select_1 = radar_select_1[
                        (radar_select_1['gdalt'] >= altitude_window_i) & (radar_select_1['gdalt'] <= altitude_window_f)]


                # second one

                # define time window
                time_window_i = gf['dates'][gf.index[np.argwhere(gf.index == pass_idx)[0][0]]] - datetime.timedelta(
                    minutes=15)
                time_window_f = gf['dates'][gf.index.max()] + datetime.timedelta(minutes=15)

                radar_select_2 = radar[(radar['dates'] >= time_window_i) & (radar['dates'] <= time_window_f)]
                gf_select_2 = gf[(gf['dates'] >= time_window_i) & (gf['dates'] <= time_window_f)]

                if len(radar_select_2) > 0:
                    Re = earth_radius(radar_lat)
                    altitude = np.mean(gf_select_2['Radius'] * 0.001 - Re) # 0.001 convert m to km

                    altitude_window_i = altitude - altitude_window
                    altitude_window_f = altitude + altitude_window

                    radar_select_2 = radar_select_2[
                        (radar_select_2['gdalt'] >= altitude_window_i) & (radar_select_2['gdalt'] <= altitude_window_f)]

#     del radar
#     del gf

    return radar_select_0,gf_select_0,radar_select_1,gf_select_1,radar_select_2,gf_select_2,radar_metadata

# radar_metadata[radar_metadata['name'] == 'Cedar file name']['value']
# gf_file.split('/')[-1]

def calc_conjunctions(radar_select, gf_select,Re):
    radar_select['nel'] = np.log10(radar_select['ne'])
    gf_select['Relative_Nel'] = np.log10(gf_select['Relative_Ne'])

    radar_mean = radar_select.groupby(['gdalt']).mean()

    gf_alt = np.mean(gf_select['Radius'] * 0.001 - Re)
    gf_nel = np.mean(gf_select['Relative_Nel'])


    x = radar_mean['nel'].values
    y = radar_mean.index.values

    # curve fit
    popt, _ = curve_fit(objective, x, y, maxfev=10000)
    a, b = popt
    #         print(popt)

    # define a sequence of inputs between the smallest and largest known inputs
    x_line = np.arange(min(x), max(x), 10e-6)
    # calculate the output for the range
    y_line = objective(x_line, a, b)

    idx = find_nearest_idx(y_line,gf_alt)
    radar_alt = y_line[idx]
    radar_nel = x_line[idx]
    plt.scatter(radar_nel, gf_nel)
    return radar_nel,gf_nel

# fig, ax = plt.subplots(figsize=(10,10))
# plt.plot([0,15],[0,15], color='k')
# plt.xlim(0,15)
# plt.ylim(0,15)

# # n=0
# names = []
# for i in iter(range(0,len(radar_list))):

#     try:
#         radar_select_0,gf_select_0,radar_select_1,gf_select_1,radar_select_2,gf_select_2,radar_metadata = find_conjunctions(radar_file= radar_list[i], gf_list = gf_list, gf_list_datetimes = gf_list_datetimes)


#         if (len(radar_select_0) > 0) & (len(gf_select_0) > 0):
#             # print('0 GF passage ', gf_select_0['dates'][gf_select_0.index == gf_select_0.index.min()].values, '->', gf_select_0['dates'][gf_select_0.index == gf_select_1.index.max()].values)
#             # print('gf : ', len(gf_select_0))
#             # print('radar : ',len(np.unique(radar_select_0['recno'])))
#             # print('------')
#             print(radar_metadata[radar_metadata['name'] == 'instrument']['value'][0])
# #             radar_nel_0,gf_nel_0 = calc_conjunctions(radar_select_0, gf_select_0, Re)
#             # n = n+1

#         if (len(radar_select_1) > 0) & (len(gf_select_1) > 0):
#             # print('1 GF passage ', gf_select_1['dates'][gf_select_1.index == gf_select_1.index.min()].values, '->', gf_select_1['dates'][gf_select_1.index == gf_select_1.index.max()].values)
#             # print('gf : ', len(gf_select_1))
#             # print('radar : ',len(np.unique(radar_select_1['recno'])))
#             # print('------')
#             print(radar_metadata[radar_metadata['name'] == 'instrument']['value'][0])
# #             radar_nel_1,gf_nel_1 =calc_conjunctions(radar_select_1, gf_select_1, Re)
#             # n = n + 1

#         if (len(radar_select_2) > 0) & (len(gf_select_2) > 0):
#             # print('2 GF passage ', gf_select_2['dates'][gf_select_2.index == gf_select_2.index.min()].values, '->', gf_select_2['dates'][gf_select_2.index == gf_select_2.index.max()].values)
#             # print('gf : ', len(gf_select_2))
#             # print('radar : ', len(np.unique(radar_select_2['recno'])))
#             # print('------')
#             print(radar_metadata[radar_metadata['name'] == 'instrument']['value'][0])
# #             radar_nel_2,gf_nel_2 =calc_conjunctions(radar_select_2, gf_select_2, Re)
#                     # n = n + 1

#     except:
#         pass

# # plt.show()
# # # print(i)
# # names = pd.DataFrame(data={"name": names})
# # names.to_csv('EISCATSvalbardISRadarLongyearbyen.csv')


/Users/andyara/opt/anaconda3/envs/gfz_spacepy/lib/python3.8/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


In [12]:
gf_list_datetimes[-100]

'2020-09-23'

In [14]:
gf = pd.read_csv(gf_list[-100], sep='\s+', header=0, index_col=False, names=headerlist)
gf['dates'] = (Time(gf['CDF Epoch'].values, format='cdf_epoch')).datetime

In [15]:
gf

,CDF Epoch,GPS,Latitude,Longitude,Radius,Latitude_QD,Longitude_QD,MLT,GRACE_1_Position,GRACE_2_Position,Iono_Corr,Distance,Relative_Hor_TEC,Relative_Ne,dates
0,63768038382000,1284854400,-43.710,121.445,6874637.73,-56.318,-165.981,8.19,-2624872.1,4288902.4,-4688678.5,-2559896.3,4190172.9,-4812165.4,2020-09-22 23:59:42
1,63768038387000,1284854405,-43.393,121.435,6874677.86,-55.963,-165.983,8.20,-2637605.2,4311471.3,-4660810.1,-2573023.2,4213301.1,-4784950.2,2020-09-22 23:59:47
2,63768038392000,1284854410,-43.076,121.424,6874717.91,-55.608,-165.986,8.20,-2650241.5,4333918.0,-4632798.7,-2586055.0,4236310.5,-4757588.2,2020-09-22 23:59:52
3,63768038397000,1284854415,-42.759,121.414,6874757.90,-55.253,-165.989,8.20,-2662780.7,4356241.9,-4604645.1,-2598991.2,4259200.3,-4730080.2,2020-09-22 23:59:57
4,63768038402000,1284854420,-42.442,121.403,6874797.82,-54.897,-165.993,8.20,-2675222.5,4378442.1,-4576350.2,-2611831.5,4281969.7,-4702427.0,2020-09-23 00:00:02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17150,63768124757000,1284940775,37.417,122.178,6877850.61,31.712,-164.053,8.32,-2882939.8,4578645.9,4246948.4,-2935287.6,4668476.0,4111178.3,2020-09-23 23:59:17
17151,63768124762000,1284940780,37.734,122.166,6877823.31,32.043,-164.050,8.32,-2869436.1,4559276.1,4276782.9,-2922148.7,4649759.6,4141591.1,2020-09-23 23:59:22
17152,63768124767000,1284940785,38.051,122.154,6877795.83,32.374,-164.047,8.32,-2855859.3,4539757.6,4306486.2,-2908934.4,4630892.1,4171876.8,2020-09-23 23:59:27
17153,63768124772000,1284940790,38.367,122.142,6877768.15,32.705,-164.044,8.32,-2842209.8,4520090.9,4336057.5,-2895645.4,4611873.8,4202034.6,2020-09-23 23:59:32


In [2]:
radars = glob.glob('../data/external/Madrigal/2018-2021/*')


In [3]:
radars[2]

'../data/external/Madrigal/2018-2021/MillstoneHillISRadar'

In [ ]:
radar_list = (glob.glob('../data/external/Madrigal/2018-2021/' + radars[2].split('/')[-1] + '/*'))
names = []

for i in iter(range(0, len(radar_list))):

    try:
        radar_select_0,gf_select_0,radar_select_1,gf_select_1,radar_select_2,gf_select_2,radar_metadata, Re = find_conjunctions(radar_file= radar_list[i], gf_list = gf_list, gf_list_datetimes = gf_list_datetimes)

        if (len(radar_select_0) > 0) & (len(gf_select_0) > 0):
            print('0 GF passage ', gf_select_0['dates'][gf_select_0.index == gf_select_0.index.min()].values, '->', gf_select_0['dates'][gf_select_0.index == gf_select_1.index.max()].values)
            # print('gf : ', len(gf_select_0))
            # print('radar : ',len(np.unique(radar_select_0['recno'])))
            # print('------')
#             print(radar_metadata[radar_metadata['name'] == 'instrument']['value'][0])
            # radar_nel_0,gf_nel_0 = calc_conjunctions(radar_select_0, gf_select_0, Re)
            # n = n+1

        if (len(radar_select_1) > 0) & (len(gf_select_1) > 0):
            print('1 GF passage ', gf_select_1['dates'][gf_select_1.index == gf_select_1.index.min()].values, '->', gf_select_1['dates'][gf_select_1.index == gf_select_1.index.max()].values)
            # print('gf : ', len(gf_select_1))
            # print('radar : ',len(np.unique(radar_select_1['recno'])))
            # print('------')
#             print(radar_metadata[radar_metadata['name'] == 'instrument']['value'][0])
            # radar_nel_1,gf_nel_1 =calc_conjunctions(radar_select_1, gf_select_1, Re)
            # n = n + 1

        if (len(radar_select_2) > 0) & (len(gf_select_2) > 0):
            print('2 GF passage ', gf_select_2['dates'][gf_select_2.index == gf_select_2.index.min()].values, '->', gf_select_2['dates'][gf_select_2.index == gf_select_2.index.max()].values)
            # print('gf : ', len(gf_select_2))
            # print('radar : ', len(np.unique(radar_select_2['recno'])))
            # print('------')
#             print(radar_metadata[radar_metadata['name'] == 'instrument']['value'][0])
            # radar_nel_2,gf_nel_2 =calc_conjunctions(radar_select_2, gf_select_2, Re)
                    # n = n + 1

    except:
        pass